#4 Lane Prediction through live cameras
Note:
1. Here we used 4 videos as live feed rather than using 4 cameras since 
we were unable to get 4 cameras and connect it to a single PC.
2. Our code not only works for 4 lanes but also works for N number of lanes.

* Importing necessary Libraries and loading our model goes here.
* We used YOLOModel for ambulance detection and VGG16 for traffic density prediction




In [41]:
import tensorflow as tf
from tensorflow.keras.models import load_model

import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout
import struct
from keras.layers import Conv2D
from keras.models import Model
from numpy import expand_dims
from keras.models import load_model
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from skimage import io
from PIL import Image 
from keras.preprocessing import image
from datetime import datetime
import os
import time
from threading import Thread
modelyolo = load_model('/content/drive/My Drive/YOLOModel.h5')   #loading YOLO Model

model =VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet') #initializing VGG16


regularizer = tf.keras.regularizers.l2(0.01)
#Adding regularizer to overcome Overfitting of the model
for layer in model.layers:
    for attr in ['kernel_regularizer']:
        if hasattr(layer, attr):
          setattr(layer, attr, regularizer)


for layer in model.layers[:11]:
  layer.trainable=False


Model = Sequential()
for layer in model.layers[:11]:
  layer.trainable=True
  Model.add(layer)
c=0  
for layer in model.layers[11:]:
  Model.add(layer)
  c+=1
  if c%4==0:
    Model.add(Dropout(0.3)) 

Model.add(GlobalAveragePooling2D())
Model.add(Dense(units=4098,activation="relu"))
Model.add(Dense(units=4098,activation="relu"))
Model.add(Dense(units=3, activation="softmax"))
Model.load_weights('/content/drive/My Drive/Traffic_Densen(VGG)(1)new.h5')
#loading weights(trained in another notebook) to the VGG16 model which we had defined above.  

modelresnet =tf.keras.applications.ResNet101()
#loading pretrained ResNet101 model

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


* Functions required for Ambulance detection

In [57]:
#functions required for V3
from keras.preprocessing import image
class BoundBox:
	def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
		self.xmin = xmin
		self.ymin = ymin
		self.xmax = xmax
		self.ymax = ymax
		self.objness = objness
		self.classes = classes
		self.label = -1
		self.score = -1
 
	def get_label(self):
		if self.label == -1:
			self.label = np.argmax(self.classes)
 
		return self.label
 
	def get_score(self):
		if self.score == -1:
			self.score = self.classes[self.get_label()]
 
		return self.score
 
def _sigmoid(x):
	return 1. / (1. + np.exp(-x))
 
def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
	grid_h, grid_w = netout.shape[:2]
	nb_box = 3
	netout = netout.reshape((grid_h, grid_w, nb_box, -1))
	nb_class = netout.shape[-1] - 5
	boxes = []
	netout[..., :2]  = _sigmoid(netout[..., :2])
	netout[..., 4:]  = _sigmoid(netout[..., 4:])
	netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
	netout[..., 5:] *= netout[..., 5:] > obj_thresh
 
	for i in range(grid_h*grid_w):
		row = i / grid_w
		col = i % grid_w
		for b in range(nb_box):
			# 4th element is objectness score
			objectness = netout[int(row)][int(col)][b][4]
			if(objectness.all() <= obj_thresh): continue
			# first 4 elements are x, y, w, and h
			x, y, w, h = netout[int(row)][int(col)][b][:4]
			x = (col + x) / grid_w # center position, unit: image width
			y = (row + y) / grid_h # center position, unit: image height
			w = anchors[2 * b + 0] * np.exp(w) / net_w # unit: image width
			h = anchors[2 * b + 1] * np.exp(h) / net_h # unit: image height
			# last elements are class probabilities
			classes = netout[int(row)][col][b][5:]
			box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes)
			boxes.append(box)
	return boxes
 
def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
	new_w, new_h = net_w, net_h
	for i in range(len(boxes)):
		x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
		y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
		boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
		boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
		boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
		boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)
 
def _interval_overlap(interval_a, interval_b):
	x1, x2 = interval_a
	x3, x4 = interval_b
	if x3 < x1:
		if x4 < x1:
			return 0
		else:
			return min(x2,x4) - x1
	else:
		if x2 < x3:
			 return 0
		else:
			return min(x2,x4) - x3
 
def bbox_iou(box1, box2):
	intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
	intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
	intersect = intersect_w * intersect_h
	w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
	w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
	union = w1*h1 + w2*h2 - intersect
	return float(intersect) / union
 
def do_nms(boxes, nms_thresh):
	if len(boxes) > 0:
		nb_class = len(boxes[0].classes)
	else:
		return
	for c in range(nb_class):
		sorted_indices = np.argsort([-box.classes[c] for box in boxes])
		for i in range(len(sorted_indices)):
			index_i = sorted_indices[i]
			if boxes[index_i].classes[c] == 0: continue
			for j in range(i+1, len(sorted_indices)):
				index_j = sorted_indices[j]
				if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
					boxes[index_j].classes[c] = 0
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck",
	"boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
	"bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
	"backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
	"sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
	"tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana",
	"apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
	"chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse",
	"remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
	"book", "clock", "vase", "scissors", "teddy bear", "hair drier","toothbrush"] 
# load and prepare an image
def load_image_pixels(filename, shape):
	# load the image to get its shape
	#image = load_img(filename)
	width, height = image.size
	cv2.imread()
	# load the image with the required size
	image = load_img(filename, target_size=shape)
	# convert to numpy array
	image = img_to_array(image)
	# scale pixel values to [0, 1]
	image = image.astype('float32')
	image /= 255.0
	# add a dimension so that we have one sample
	image = expand_dims(image, 0)
	return image, width, height
 
# get all of the results above a threshold
def get_boxes(boxes, labels, thresh):
	v_boxes, v_labels, v_scores = list(), list(), list()
	# enumerate all boxes
	for box in boxes:
		# enumerate all possible labels
		for i in range(len(labels)):
			# check if the threshold for this label is high enough
			if box.classes[i] > thresh:
				v_boxes.append(box)
				v_labels.append(labels[i])
				v_scores.append(box.classes[i]*100)
				# don't break, many labels may trigger for one box
	return v_boxes, v_labels, v_scores
 
# draw all results
def draw_boxes(filename, v_boxes, v_labels, v_scores):
	# load the image
	data = pyplot.imread(filename)
	# plot the image
	pyplot.imshow(data)
	# get the context for drawing boxes
	ax = pyplot.gca()
	# plot each box
	for i in range(len(v_boxes)):
		box = v_boxes[i]
		# get coordinates
		y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
		# calculate width and height of the box
		width, height = x2 - x1, y2 - y1
		# create the shape
		rect = Rectangle((x1, y1), width, height, fill=False, color='white')
		# draw the box
		ax.add_patch(rect)
		# draw text and score in top left corner
		label = "%s (%.3f)" % (v_labels[i], v_scores[i])
		pyplot.text(x1, y1, label, color='white')
	# show the plot
	pyplot.show()
 

* Below function returns True if an ambulance is found in an image else it returns False

In [60]:
def checkambulance(photo_filename):#PASS IMAGE PATH AS ARGUMENT
  input_w, input_h = 416, 416
  from keras.preprocessing.image import image
  # load and prepare image
  image, image_w, image_h = load_image_pixels(photo_filename, (input_w, input_h))
  # make prediction
  
  yhat = modelyolo.predict(image)
  anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
  class_threshold = 0.6
  boxes = list()
  for i in range(len(yhat)):
    # decode the output of the network
    boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)
  # correct the sizes of the bounding boxes for the shape of the image
  correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
  # suppress non-maximal boxes
  do_nms(boxes, 0.5)
  
  # get the details of the detected objects
  v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
  #im=io.imread(photo_filename)
  for i in range(len(v_boxes)):
    if(v_labels[i]=='truck' or v_labels[i]=='bus'):
      box=v_boxes[i]
      y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax      #EXTRACT CO-ORDINATES OF BOXES
      width, height = x2 - x1, y2 - y1
      if(height==0):
        height=1
      image1 =im[y1:y1+height,x1:x1+width]      #CROP BOX AND FEED TO ANOTHER MODEL
      image1 = cv2.resize(image1,(224,224), interpolation = cv2.INTER_AREA)
      img = np.expand_dims(image1, axis=0)
      result=modelresnet.predict(img)
      label = decode_predictions(result,top=15)
      for i in range(15):
        if(label[0][i][1]=='ambulance'):
          plt.imshow(image1)
          plt.show() 
          return True
  return False

Below code does the following
 *  Takes Frames from the videos provided
 *  Predicts the Traffic density for each Lane
 *  Amount of time for green signal will be given accordingly

In [48]:
def get_actual(a): ###actually our model returns 0 for light 1 for heavy and 2 for moderate
  if a==0:        ###to convert it into a meaningful way we are swapping all the values and considering as follows:
    return 2          ###   0 for light, 1 for moderate, 2 for heavy >>this is meaningful than the previous
  if a==1:
    return 0
  if a==2:
    return 1

In [61]:
# This code is commented because we were working in colab and colab doesn't support 
#         accessing the data from the database that is residing in our local laptop

# This function will work if this code runs locally rather than on cloud

# If all the lanes are heavy, then the below function adds to the database and notify users
#                                                        regarding the heavy traffic at the signal junction
'''def add_to_database(inname,indesc):
  mydb = mysql.connector.connect(
            host='127.0.0.1',
            database='sitms',
            user='root',
            password='',
            use_pure=True     
            )
  nowtime =str(datetime.now().time())
  nowdate = str(datetime.now().date())
  mycursor = mydb.cursor() 
  sql = "INSERT INTO sitms_report_incident (incident_name,incident_desc,incident_date,incident_time,incident_location,incident_user) VALUES (%s, %s,%s,%s,%s,%s)"
  inplace='Signal Junction'
  inuser='Traffic Signal'
  val = inname,indesc,nowdate,nowtime,inplace,inuser
  mycursor.execute(sql, val)
  mydb.commit()'''

# This class creates lane objects with a set of attributes (laneno, traffic_mode[high or medium or low], presence of ambulance[True or False])
class Lane:
  def __init__(self,laneno,traffic_mode,ambulance):
    self.laneno=laneno
    self.ambulance=ambulance
    self.traffic_mode=traffic_mode
class Traffic_manager:
  @staticmethod
  def return_priority(tensor):  # returns most prior lane. ex: if ambulance is present in a lane, then that is the most prior lane
    for lane in tensor:
      if lane.ambulance:
        return lane
    priority=max(tensor,key=lambda l:l.traffic_mode)
    return priority

  @staticmethod
  def traffic_signal(lane,laneno,emergency=False):   # returns amount of time to be given for a given lane
    if emergency:
      print('green for lane'+str(laneno)+'due to emergency')
      time.sleep(4)
    else:
      print('green for lane'+str(laneno),end=' ')
      if lane.traffic_mode==0:
        print('time alloted is 1sec','traffic mode is',lane.traffic_mode)
        time.sleep(1)
      if lane.traffic_mode==1:
        print('time alloted is 3sec','traffic mode is',lane.traffic_mode)
        time.sleep(3)
      if lane.traffic_mode==2:
        print('time alloted is 5sec','traffic mode is',lane.traffic_mode)
        time.sleep(5)

  @staticmethod
  def cycle(tensor):
    lanes=[i.laneno for i in tensor]
    while tensor:
      prior=Traffic_manager.return_priority(tensor)
      arguments=[prior,prior.laneno,prior.ambulance==True]
      signalthread=Thread(target=Traffic_manager.traffic_signal,args=arguments)
      signalthread.start()    #here we are using threads so that CPU will be doing predictions for next instance while the traffic_signal func is in sleep
      if(prior.ambulance==False):
        tensor.remove(prior)
        lanes.remove(prior.laneno)
      
      tensor=Traffic_manager.predictions(z)
      tensor=[lane for lane in tensor if lane.laneno in lanes]
      signalthread.join()

  @staticmethod
  def predictions(images):
    p={}
    for i in range(len(images)):
      img=images[i]
      outcome=list(Model.predict_classes(img))
      outcome[0]=get_actual(outcome[0])
      p[i]=outcome
      check=False   #this is the output of check_ambulance function. 
      p[i]+=[check] #ex: p would be p={1:[high,false]},boolean represents presence of ambulance
    array=[] #this array consists of Lane objects
    for i in p:
      array.append(Lane(i,p[i][0],p[i][1]))
    return array


**Below code is Driver code which takes 4 videos as input and gives specified time for each lane accordingly**

In [62]:
import cv2
v1=cv2.VideoCapture('/content/drive/My Drive/final video clips/VID_20200730_150201.mp4')
# v2=cv2.VideoCapture('/content/drive/My Drive/final video clips/VID_20200730_151106.mp4')
v3=cv2.VideoCapture('/content/drive/My Drive/final video clips/VID_20200730_151416.mp4')
v4=cv2.VideoCapture('/content/drive/My Drive/final video clips/VID_20200730_151545.mp4')
count=0
s1,s2,s3,s4=[1,1,1,1]
z=[]
while s1 and s2  and s4:
  s1,i1=v1.read()
  # s2,i2=v2.read()
  s3,i3=v3.read()
  s4,i4=v4.read()
  if count%50==0:   #50 defines that taking frame for every 50 frames
    z=[i1,i3,i4]
    for i in range(len(z)):
      z[i]=np.resize(z[i],(224,224,3))
      z[i]=np.expand_dims(z[i],axis=0)
    result=Traffic_manager.predictions(z)
    print(count//50,'th time')
    Traffic_manager.cycle(result)
  count+=1

### Here we are interupting the while loop instead of going through entire video

0 th time
green for lane0 time alloted is 1sec traffic mode is 0
green for lane1 time alloted is 1sec traffic mode is 0
green for lane2 time alloted is 1sec traffic mode is 0
1 th time
green for lane0 time alloted is 3sec traffic mode is 1


KeyboardInterrupt: ignored

# Please donot think that the upper block is an error.  We are just terminating the  infinite while loop instead of going through the entire video.